# UrbanSound8K Audio Classification - Data Preprocessing

This notebook preprocesses the UrbanSound8K dataset for deep learning classification.
It extracts features from audio files and creates train/val/test splits for 10-fold CV.

**Output:**
Creates `preprocessed_data/` folder with:
- MFCC features for CNN (compact, ~2GB)
- Mel spectrograms for CNN alternative
- Raw audio for RNN (larger, ~8GB)
- All 10 folds ready for cross-validation

### Setup and Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from pathlib import Path
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

### Configuration

Update `DATASET_PATH` to point to your UrbanSound8K dataset location.

In [ ]:
# CONFIGURATION
DATASET_PATH = "C:\\Users\\leono\\OneDrive\\Desktop\\AC2_folder\\UrbanSound8K"

# Audio processing parameters
TARGET_SR = 22050        # Target sampling rate (Hz)
DURATION = 4.0          # Target duration (seconds)
N_MFCC = 40            # Number of MFCC coefficients
N_FFT = 2048           # FFT window size
HOP_LENGTH = 512       # Hop length for STFT
N_MELS = 128           # Number of mel frequency bins

# What to extract (set to False to skip and save space)
EXTRACT_MFCC = True          # ~2GB - RECOMMENDED for CNN
EXTRACT_MEL_SPEC = True      # ~2GB - Alternative for CNN
EXTRACT_RAW = False          # ~8GB - For RNN (optional, large file)

# Data augmentation (only applied to training data)
AUGMENT_TRAINING = True

# Output directory
OUTPUT_DIR = "./preprocessed_data"

### Preprocessing Functions

In [ ]:
def load_and_normalize_audio(filepath: str, augment: bool = False) -> np.ndarray:
    """
    Load audio file and normalize to fixed length and sampling rate.
    
    Args:
        filepath: Path to audio file
        augment: Whether to apply data augmentation
        
    Returns:
        Normalized audio array of shape (TARGET_SR * DURATION,)
    """
    try:
        # Load audio
        audio, sr = librosa.load(filepath, sr=TARGET_SR)
        
        # Apply augmentation if requested (training only)
        if augment:
            aug_type = np.random.randint(0, 4)
            if aug_type == 0:
                # Time stretching
                rate = np.random.uniform(0.8, 1.2)
                audio = librosa.effects.time_stretch(audio, rate=rate)
            elif aug_type == 1:
                # Pitch shifting
                n_steps = np.random.randint(-2, 3)
                audio = librosa.effects.pitch_shift(audio, sr=TARGET_SR, n_steps=n_steps)
            elif aug_type == 2:
                # Add noise
                noise = np.random.normal(0, 0.005, audio.shape)
                audio = audio + noise
        
        # Normalize length (pad or truncate to DURATION seconds)
        target_length = int(TARGET_SR * DURATION)
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
        else:
            audio = audio[:target_length]
        
        # Normalize amplitude
        audio = librosa.util.normalize(audio)
        
        return audio
        
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return np.zeros(int(TARGET_SR * DURATION))

In [ ]:
def extract_mfcc(audio: np.ndarray) -> np.ndarray:
    """Extract MFCC features from audio. Shape: (N_MFCC, time_frames)"""
    mfcc = librosa.feature.mfcc(
        y=audio, sr=TARGET_SR, n_mfcc=N_MFCC, 
        n_fft=N_FFT, hop_length=HOP_LENGTH
    )
    return mfcc

In [ ]:
def extract_mel_spectrogram(audio: np.ndarray) -> np.ndarray:
    """Extract mel spectrogram from audio. Shape: (N_MELS, time_frames)"""
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=TARGET_SR, n_fft=N_FFT, 
        hop_length=HOP_LENGTH, n_mels=N_MELS
    )
    # Convert to log scale (dB)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

In [ ]:
def process_fold(metadata: pd.DataFrame, 
                 fold_num: int, 
                 split_type: str,
                 dataset_path: Path,
                 augment: bool = False) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Process all audio files for a specific fold and split.
    
    Args:
        metadata: DataFrame with file information
        fold_num: Fold number to process
        split_type: 'train', 'val', or 'test'
        dataset_path: Path to UrbanSound8K dataset
        augment: Whether to apply augmentation
        
    Returns:
        features_dict: Dictionary with different feature types
        labels: Array of class labels
        info: Processing information
    """
    features = {'mfcc': [], 'mel_spec': [], 'raw': []}
    labels = []
    
    fold_data = metadata[metadata['fold'] == fold_num]
    print(f"  Processing {split_type} fold {fold_num}: {len(fold_data)} files")
    
    for idx, row in fold_data.iterrows():
        # Construct audio file path
        audio_path = dataset_path / "audio" / f"fold{row['fold']}" / row['slice_file_name']
        
        if not audio_path.exists():
            print(f"    Warning: {audio_path} not found, skipping")
            continue
        
        # Load and normalize audio
        audio = load_and_normalize_audio(str(audio_path), augment=augment)
        
        # Extract features based on configuration
        if EXTRACT_MFCC:
            features['mfcc'].append(extract_mfcc(audio))
        if EXTRACT_MEL_SPEC:
            features['mel_spec'].append(extract_mel_spectrogram(audio))
        if EXTRACT_RAW:
            features['raw'].append(audio)
        
        # Store label
        labels.append(row['classID'])
    
    # Convert lists to arrays
    for key in features:
        if features[key]:
            features[key] = np.array(features[key])
    
    labels = np.array(labels)
    
    info = {
        'split': split_type,
        'fold': fold_num,
        'n_samples': len(labels)
    }
    
    return features, labels, info

In [ ]:
def create_10fold_splits(metadata: pd.DataFrame) -> list:
    """
    Create 10-fold cross-validation split configurations.
    Each fold gets to be test once, with another fold as validation.
    
    Returns:
        List of tuples: (test_fold, val_fold)
    """
    splits = []
    for test_fold in range(1, 11):
        # Use the next fold as validation (wrap around if needed)
        val_fold = (test_fold % 10) + 1
        splits.append((test_fold, val_fold))
    return splits

### Main Preprocessing Function

In [ ]:
def preprocess_dataset():
    """
    Main preprocessing function - processes entire dataset and saves to disk.
    """
    print("="*70)
    print("UrbanSound8K Data Preprocessing")
    print("="*70)
    
    # Load metadata
    dataset_path = Path(DATASET_PATH)
    metadata_path = dataset_path / "metadata" / "UrbanSound8K.csv"
    
    if not metadata_path.exists():
        print(f"ERROR: Metadata file not found at {metadata_path}")
        print("Please update DATASET_PATH in the script.")
        return
    
    metadata = pd.read_csv(metadata_path)
    print(f"\nLoaded metadata: {len(metadata)} audio files")
    print(f"Classes: {sorted(metadata['class'].unique())}")
    
    # Create class mapping
    class_to_idx = {cls: idx for idx, cls in enumerate(sorted(metadata['class'].unique()))}
    idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}
    
    # Get 10-fold CV splits
    cv_splits = create_10fold_splits(metadata)
    
    # Create output directory
    output_path = Path(OUTPUT_DIR)
    output_path.mkdir(parents=True, exist_ok=True)
    
    print(f"\nConfiguration:")
    print(f"  Target SR: {TARGET_SR} Hz")
    print(f"  Duration: {DURATION} seconds")
    print(f"  Extracting MFCC: {EXTRACT_MFCC}")
    print(f"  Extracting Mel Spectrogram: {EXTRACT_MEL_SPEC}")
    print(f"  Extracting Raw Audio: {EXTRACT_RAW}")
    print(f"  Augmentation: {AUGMENT_TRAINING}")
    print(f"  Output: {output_path}")
    
    # Process each fold
    for cv_idx, (test_fold, val_fold) in enumerate(cv_splits, 1):
        print(f"\n{'='*70}")
        print(f"Processing CV Split {cv_idx}/10")
        print(f"  Test fold: {test_fold}, Val fold: {val_fold}, Train folds: others")
        print(f"{'='*70}")
        
        # Get train folds (all except test and val)
        train_folds = [f for f in range(1, 11) if f not in [test_fold, val_fold]]
        
        # Process training data
        print("\n[1/3] Training data:")
        train_features_all = {'mfcc': [], 'mel_spec': [], 'raw': []}
        train_labels_all = []
        
        for train_fold in train_folds:
            features, labels, _ = process_fold(
                metadata, train_fold, 'train', dataset_path, 
                augment=AUGMENT_TRAINING
            )
            
            # Accumulate features from all training folds
            for key in train_features_all:
                if key in features and len(features[key]) > 0:
                    train_features_all[key].append(features[key])
            train_labels_all.append(labels)
        
        # Concatenate all training folds
        for key in train_features_all:
            if train_features_all[key]:
                train_features_all[key] = np.concatenate(train_features_all[key], axis=0)
            else:
                train_features_all[key] = None
        train_labels_all = np.concatenate(train_labels_all, axis=0)
        
        # Process validation data
        print("\n[2/3] Validation data:")
        val_features, val_labels, _ = process_fold(
            metadata, val_fold, 'val', dataset_path, augment=False
        )
        
        # Process test data
        print("\n[3/3] Test data:")
        test_features, test_labels, _ = process_fold(
            metadata, test_fold, 'test', dataset_path, augment=False
        )
        
        # Save to disk
        fold_dir = output_path / f"fold_{cv_idx}"
        fold_dir.mkdir(exist_ok=True)
        
        # Save each feature type
        for feature_type in ['mfcc', 'mel_spec', 'raw']:
            if train_features_all[feature_type] is not None:
                feature_dir = fold_dir / feature_type
                feature_dir.mkdir(exist_ok=True)
                
                np.save(feature_dir / "X_train.npy", train_features_all[feature_type])
                np.save(feature_dir / "X_val.npy", val_features[feature_type])
                np.save(feature_dir / "X_test.npy", test_features[feature_type])
        
        # Save labels (same for all feature types)
        np.save(fold_dir / "y_train.npy", train_labels_all)
        np.save(fold_dir / "y_val.npy", val_labels)
        np.save(fold_dir / "y_test.npy", test_labels)
        
        print(f"\n  Saved fold {cv_idx}:")
        if EXTRACT_MFCC:
            print(f"    MFCC - Train: {train_features_all['mfcc'].shape}, "
                  f"Val: {val_features['mfcc'].shape}, Test: {test_features['mfcc'].shape}")
        if EXTRACT_MEL_SPEC:
            print(f"    Mel Spec - Train: {train_features_all['mel_spec'].shape}, "
                  f"Val: {val_features['mel_spec'].shape}, Test: {test_features['mel_spec'].shape}")
        if EXTRACT_RAW:
            print(f"    Raw - Train: {train_features_all['raw'].shape}, "
                  f"Val: {val_features['raw'].shape}, Test: {test_features['raw'].shape}")
    
    # Save metadata
    metadata_dict = {
        'target_sr': TARGET_SR,
        'duration': DURATION,
        'n_mfcc': N_MFCC,
        'n_mels': N_MELS,
        'class_to_idx': class_to_idx,
        'idx_to_class': idx_to_class,
        'num_classes': len(class_to_idx)
    }
    np.save(output_path / "metadata.npy", metadata_dict, allow_pickle=True)
    
    print(f"\n{'='*70}")
    print("PREPROCESSING COMPLETE!")
    print(f"{'='*70}")
    print(f"\nData saved to: {output_path}")
    print(f"Total folds: 10")
    print("\nShare the '{OUTPUT_DIR}'")
    print("\nData can be loaded using the companion notebook or:")
    print("  X_train = np.load('preprocessed_data/fold_1/mfcc/X_train.npy')")
    print("  y_train = np.load('preprocessed_data/fold_1/y_train.npy')")

### Utility Functions for Loading Data

In [ ]:
def load_fold(fold_num: int, feature_type: str = 'mfcc', 
              data_dir: str = './preprocessed_data') -> Tuple:
    """
    Load preprocessed data for a specific fold.
    
    Args:
        fold_num: Fold number (1-10)
        feature_type: 'mfcc', 'mel_spec', or 'raw'
        data_dir: Path to preprocessed data directory
        
    Returns:
        X_train, y_train, X_val, y_val, X_test, y_test
        
    Example:
        X_train, y_train, X_val, y_val, X_test, y_test = load_fold(1, 'mfcc')
    """
    data_path = Path(data_dir)
    fold_path = data_path / f"fold_{fold_num}"
    
    X_train = np.load(fold_path / feature_type / "X_train.npy")
    X_val = np.load(fold_path / feature_type / "X_val.npy")
    X_test = np.load(fold_path / feature_type / "X_test.npy")
    
    y_train = np.load(fold_path / "y_train.npy")
    y_val = np.load(fold_path / "y_val.npy")
    y_test = np.load(fold_path / "y_test.npy")
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
def load_metadata(data_dir: str = './preprocessed_data') -> Dict:
    """
    Load preprocessing metadata.
    
    Returns:
        Dictionary with class mappings and parameters
    """
    metadata = np.load(Path(data_dir) / "metadata.npy", allow_pickle=True).item()
    return metadata

### Run Preprocessing

Execute the cell below to start the preprocessing pipeline.

In [ ]:
# Check if dataset path has been updated
if DATASET_PATH == "/path/to/UrbanSound8K":
    print("ERROR: Please update DATASET_PATH in the configuration cell before running!")
    print("Set it to the location of your UrbanSound8K folder.")
else:
    # Run preprocessing
    preprocess_dataset()

### Example: Load and Verify Preprocessed Data

In [ ]:
# Load fold 1 data as an example
try:
    X_train, y_train, X_val, y_val, X_test, y_test = load_fold(1, 'mfcc')
    metadata = load_metadata()
    
    print("Successfully loaded fold 1 data:")
    print(f"  Training set: {X_train.shape} samples")
    print(f"  Validation set: {X_val.shape} samples")
    print(f"  Test set: {X_test.shape} samples")
    print(f"\nMetadata:")
    print(f"  Number of classes: {metadata['num_classes']}")
    print(f"  Class mapping: {metadata['idx_to_class']}")
except Exception as e:
    print(f"Data not yet preprocessed. Run the preprocessing cell first.")
    print(f"Error: {e}")